<a href="https://colab.research.google.com/github/khiro45/hand-sign-recognition/blob/main/hand_signes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sign-language-detection-using-images:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5494158%2F9103414%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240822%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240822T201543Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D22f141f0c71ef77fb704065c428e77bab7f0c587326c65151f29f38abe40b7775d8d383eaebd6e587586b4ecb486de8f7ebcf5de9f2686ebebc06642387d0b3efebf387c1bbcac2d1663476b7b41f224fd5e457d9152274552b7f40a7071975837bebe34c96d9bb591e2e2728fbfcea2835270b3bffa45c61ff0fa8a6b9c3099c980f7cd0b218cbe634d37e32603d0bbef91f7bde34de1421799e32c98ae4ae99f4bcc232672c00dea77daeff78e0e07ba3b8b5032ca0bae8ab0f7138d2abbdb30934b1b61508ba2050709e38c87c6c6cee88ec6655dabb9373c87266d9ff8d0153cc470c8140a2942c97fd99a21a8da185073841cf47c965a858c851dc01fc3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 280817962 bytes downloaded
Downloaded and uncompressed: sign-language-detection-using-images
Data source import complete.


In [ ]:
from skimage.io import imread
from skimage.transform import resize
import os
data_path='/kaggle/input/sign-language-detection-using-images/data'

data=[]
labels=[]

counter=500
for category in os.listdir(data_path):
    a=1
    for file in os.listdir(os.path.join(data_path,category)):

        a+=1
        img_path = os.path.join(data_path, category, file )

        img=imread(img_path)
#         img = img.astype('float32')
        resized_img= resize(img,(32,32))

        data.append(resized_img)

        labels.append(category)
        if a>200 :
            break


print(len(data))
print(len(labels))


7000
7000


In [ ]:
x_train.shape
y_train.shape

(4690,)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

images=np.array(data)
label=np.array(labels)

Le=LabelEncoder()

encoded_label=Le.fit_transform(label)

x_train, x_test, y_train, y_test = train_test_split(
      images
    , encoded_label
    , test_size=0.33
    , random_state=42)



In [ ]:
#model creation


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense



model= Sequential()

model.add(Conv2D(32, (3,3), padding= "same", activation ='relu',input_shape=(32,32,3)))
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3,3), padding= "same", activation ='relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(35 , activation ='softmax'))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)                   │ (None, 32, 32, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 16, 16, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │         524,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 35)                  │           2,275 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 536,771 (2.05 MB)

 Trainable params: 536,771 (2.05 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
              loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
traning= model.fit(x_train , y_train , epochs= 15 , batch_size=64, validation_data=(x_test,y_test), verbose=1)

Epoch 1/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 16s 195ms/step - accuracy: 0.5393 - loss: 1.8033 - val_accuracy: 0.9974 - val_loss: 0.0290
Epoch 2/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 9s 119ms/step - accuracy: 0.9968 - loss: 0.0133 - val_accuracy: 1.0000 - val_loss: 0.0080
Epoch 3/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 122ms/step - accuracy: 0.9991 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 0.0058
Epoch 4/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 11s 134ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 0.0029
Epoch 5/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 11s 138ms/step - accuracy: 1.0000 - loss: 6.1719e-04 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 6/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 8s 112ms/step - accuracy: 1.0000 - loss: 1.0824e-04 - val_accuracy: 1.0000 - val_loss: 8.6021e-04
Epoch 7/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 12s 130ms/step - accuracy: 1.0000 - loss: 9.7900e-05 - val_accuracy: 1.0000 - val_loss: 7.4442e-04
Epoch 8/15
74/74 ━━━━━━━━━━━━━━━━━━━━ 11s 146ms/step - accuracy: 1.0000 - loss:

In [ ]:

test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


Test loss: 0.0003176859172526747
Test accuracy: 1.0
